In [ ]:
from pipeline import PrepPipe, OffensiveFormation, DefensiveClustering, StandardScalerColumns

In [186]:
prep_pipe = PrepPipe()

positional data already downloaded.
reading positional data.
returning positional data.


In [187]:
X_train, X_test, y_train, y_test = prep_pipe.clean_data()

In [188]:
off_col = prep_pipe.train_test.ofc.drop(['gameId', 'playId', 'gamePlayId', 'week'], axis=1).columns

In [189]:
def_col = prep_pipe.train_test.dfc.drop(['week', 'index'], axis=1).columns

In [190]:
off_form_cols = off_col[:-9]
off_info_cols = off_col[-9:]
off_form

In [211]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
import pandas as pd

off_form_pipe = Pipeline([('off_form_pred', OffensiveFormation())])
off_info_pipe = Pipeline([('scaler', StandardScalerColumns())])
off_pre_one_pipe = ColumnTransformer([('info', off_info_pipe, off_info_cols),
                                      ('form', off_form_pipe, off_form_cols),
                                   ])
off_pre_one_add_col = Pipeline([('off_pre_one', off_pre_one_pipe),
                               ('func_trans', FunctionTransformer(lambda x: pd.DataFrame(x,
                                columns=list(off_info_cols) + list(off_form_cols))))])
form_one_pipe = ColumnTransformer([('off_form_one', OneHotEncoder(), [-1]),
                                   ('nothing', FunctionTransformer(lambda x: x), slice(0, -1))])
off_full_pipe = Pipeline([('full_cols', off_pre_one_add_col), ('one_hot', form_one_pipe)])

def_cluster_pipe = Pipeline([('def_clust', DefensiveClustering())])
form_one_pipe = ColumnTransformer([('def_clust_one', OneHotEncoder(), [-1]),
                                   ('nothing', FunctionTransformer(lambda x: x), slice(0, -1, 1))])
def_full_pipe = Pipeline([('def_clust_full', def_cluster_pipe), ('def_clust_one', form_one_pipe)])

full_pipe = ColumnTransformer([('off', off_full_pipe, off_col), ('def', def_full_pipe, def_col)])
full_pipe_model = Pipeline([('full_pipe', full_pipe), ('model', LogisticRegression())])

In [212]:
arr = off_full_pipe.fit_transform(X_train[off_col])

Offensive formation model fitted
Offensive formation predicted


In [220]:
arr[:, 1:-2].shape

(13920, 97)

In [169]:
def_full_pipe.fit_transform(X_train[def_col])

KMeans and PCA fitted
Defensive position transformed


<13920x5 sparse matrix of type '<class 'numpy.float64'>'
	with 13920 stored elements in Compressed Sparse Row format>

In [107]:
(X_train.iloc[:, :10] - X_train.iloc[:, :10].mean(axis=0).values.reshape(1, -1)) /

,FBL0_x,FBR0_x,HBL0_x,HBL1_x,HBR0_x,HBR1_x,QB0_x,QB1_x,RBL0_x,RBL1_x
0,-5.051531,0.059902,0.095219,0.001447,0.086014,0.000468,2.541775,0.013234,2.420381,0.031098
1,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,2.671775,0.013234,-4.949619,0.031098
2,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-1.288225,0.013234,2.420381,0.031098
3,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-0.898225,0.013234,-1.819619,0.031098
4,0.068469,-4.530098,0.095219,0.001447,0.086014,0.000468,-0.608225,0.013234,1.090381,0.031098
...,...,...,...,...,...,...,...,...,...,...
13915,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-0.098225,0.013234,-2.439619,0.031098
13916,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-0.338225,0.013234,2.420381,0.031098
13917,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-0.578225,0.013234,2.420381,0.031098
13918,0.068469,0.059902,0.095219,0.001447,0.086014,0.000468,-0.258225,0.013234,2.420381,0.031098


In [104]:
X_train

,FBL0_x,FBR0_x,HBL0_x,HBL1_x,HBR0_x,HBR1_x,QB0_x,QB1_x,RBL0_x,RBL1_x,...,SSL2_y_start,SSR0_act,SSR0_x_start,SSR0_y_start,SSR1_act,SSR1_x_start,SSR1_y_start,SSR2_act,SSR2_x_start,SSR2_y_start
0,-5.12,0.00,0.0,0.0,0.0,0.0,-1.73,0.0,0.00,0.0,...,0.0,M,1.11,-9.87,0,0.00,0.00,0,0.0,0.0
1,0.00,0.00,0.0,0.0,0.0,0.0,-1.60,0.0,-7.37,0.0,...,0.0,Z,2.45,-12.48,0,0.00,0.00,0,0.0,0.0
2,0.00,0.00,0.0,0.0,0.0,0.0,-5.56,0.0,0.00,0.0,...,0.0,0,0.00,0.00,0,0.00,0.00,0,0.0,0.0
3,0.00,0.00,0.0,0.0,0.0,0.0,-5.17,0.0,-4.24,0.0,...,0.0,0,0.00,0.00,0,0.00,0.00,0,0.0,0.0
4,0.00,-4.59,0.0,0.0,0.0,0.0,-4.88,0.0,-1.33,0.0,...,0.0,M,1.23,-15.01,0,0.00,0.00,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,0.00,0.00,0.0,0.0,0.0,0.0,-4.37,0.0,-4.86,0.0,...,0.0,M,11.58,-11.14,M,5.33,-11.15,0,0.0,0.0
13916,0.00,0.00,0.0,0.0,0.0,0.0,-4.61,0.0,0.00,0.0,...,0.0,M,5.69,-10.15,M,13.78,-11.31,0,0.0,0.0
13917,0.00,0.00,0.0,0.0,0.0,0.0,-4.85,0.0,0.00,0.0,...,0.0,0,0.00,0.00,0,0.00,0.00,0,0.0,0.0
13918,0.00,0.00,0.0,0.0,0.0,0.0,-4.53,0.0,0.00,0.0,...,0.0,M,13.88,-5.15,0,0.00,0.00,0,0.0,0.0
